# Exemple d'agent de réservation d'hôtel et de vol

Cette solution vous aidera à réserver des billets d'avion et un hôtel. Le scénario est un voyage de Londres Heathrow LHR le 20 février 2024 à destination de New York JFK, avec un retour le 27 février 2025, en classe économique avec British Airways uniquement. Je souhaite séjourner dans un hôtel Hilton à New York, merci de fournir les coûts pour le vol et l'hôtel.


# Initialiser le service Azure AI Agent et obtenir les informations de configuration depuis **.env**

### **.env**

Créez un fichier .env

Le fichier **.env** contient la chaîne de connexion du service Azure AI Agent, le modèle utilisé par AOAI, ainsi que les informations correspondantes pour le service API de recherche Google, ENDPOINT, etc.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Nom de déploiement de votre modèle Azure AI Agent Service"

[**NOTE**] Vous aurez besoin d'un modèle avec une limite de 100 000 tokens par minute (Rate Limit) et une limite de 600 requêtes par minute (Request per minute).

  Vous pouvez obtenir un modèle dans Azure AI Foundry - Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Chaîne de connexion de votre projet Azure AI Agent Service"

  Vous pouvez obtenir la chaîne de connexion du projet dans l'aperçu de votre projet sur l'écran du portail AI Foundry.

- **SERPAPI_SEARCH_API_KEY** = "Votre clé API SERPAPI Search"
- **SERPAPI_SEARCH_ENDPOINT** = "Votre endpoint SERPAPI Search"

Pour obtenir le nom de déploiement du modèle et la chaîne de connexion du projet Azure AI Agent Service, vous devez créer un service Azure AI Agent. Il est recommandé d'utiliser [ce modèle](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) pour le créer directement. (***Remarque :*** Le service Azure AI Agent est actuellement limité à certaines régions. Il est conseillé de consulter [ce lien](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) pour définir la région.)

L'agent doit accéder à SERPAPI. Il est recommandé de s'inscrire via [ce lien](https://serpapi.com/searches). Après l'inscription, vous pourrez obtenir une clé API unique et un ENDPOINT.


# Configuration

Pour exécuter ce notebook, vous devez vous assurer d'avoir installé les bibliothèques nécessaires en exécutant `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Votre version de Semantic Kernel doit être au moins 1.27.2.


Chargez les paramètres et ressources de votre fichier .env, veuillez vous assurer que vous avez ajouté vos clés et paramètres et créé un fichier .env local.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Se connecter à Azure

Vous devez maintenant vous connecter à Azure. Ouvrez un terminal et exécutez la commande suivante :

```bash
az login
```

Cette commande vous invitera à saisir vos identifiants Azure, permettant ainsi au service Azure AI Agent de fonctionner correctement.


# Explication :
Ceci est une variable qui stocke la clé API permettant d'accéder à un service d'API SERP (Search Engine Results Page). Une clé API est un identifiant unique utilisé pour authentifier les requêtes associées à votre compte.

Objectif : L'objectif de cette ligne est de stocker la clé API dans une variable afin qu'elle puisse être utilisée pour authentifier les requêtes vers le service d'API SERP. La clé API est nécessaire pour accéder au service et effectuer des recherches.

Comment obtenir une clé API SERP : Pour obtenir une clé API SERP, suivez ces étapes générales sur https://serpapi.com (les étapes exactes peuvent varier en fonction du service d'API SERP spécifique que vous utilisez) :

Choisissez un service d'API SERP : Il existe plusieurs services d'API SERP disponibles, tels que SerpAPI, Google Custom Search JSON API, et d'autres. Choisissez celui qui correspond le mieux à vos besoins.

Inscrivez-vous pour un compte : Rendez-vous sur le site web du service d'API SERP choisi et inscrivez-vous pour un compte. Vous devrez peut-être fournir des informations de base et vérifier votre adresse e-mail.

Créez une clé API : Après vous être inscrit, connectez-vous à votre compte et accédez à la section API ou au tableau de bord. Recherchez une option pour créer ou générer une nouvelle clé API.  
Copiez la clé API dans votre fichier .env.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Explication :
BASE_URL : C'est une variable qui stocke l'URL de base pour le point de terminaison de l'API SERP. Le nom de la variable BASE_URL est une convention utilisée pour indiquer que cette URL est le point de départ pour effectuer des requêtes API.  
'https://serpapi.com/search' :  

C'est la chaîne URL réelle assignée à la variable BASE_URL. Elle représente le point de terminaison pour effectuer des requêtes de recherche en utilisant l'API SERP.

# Objectif :
L'objectif de cette ligne est de définir une constante qui contient l'URL de base pour l'API SERP. Cette URL sera utilisée comme point de départ pour construire des requêtes API afin d'effectuer des opérations de recherche.

# Utilisation :
En définissant l'URL de base dans une variable, vous pouvez facilement la réutiliser dans votre code chaque fois que vous devez effectuer des requêtes à l'API SERP. Cela rend votre code plus facile à maintenir et réduit le risque d'erreurs liées à la codification en dur de l'URL à plusieurs endroits. L'exemple actuel est https://serpapi.com/search?engine=bing qui utilise l'API de recherche Bing. Différentes API peuvent être sélectionnées sur https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Explication :

C'est ici que se trouve le code de votre plugin.

Définition de la classe : `class BookingPlugin` : Définit une classe nommée BookingPlugin qui contient des méthodes pour réserver des hôtels et des vols.

Méthode de réservation d'hôtel :

- `@kernel_function(description="booking hotel")` : Un décorateur qui décrit la fonction comme une fonction noyau pour la réservation d'hôtels.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:` : Définit une méthode pour réserver des hôtels avec des paramètres annotés et un type de retour.

La méthode construit un dictionnaire de paramètres pour la requête de réservation d'hôtel et envoie une requête GET à l'API SERP. Elle vérifie le statut de la réponse et retourne les propriétés de l'hôtel si la requête est réussie, ou None si la requête a échoué.

Méthode de réservation de vol :

- `@kernel_function(description="booking flight")` : Un décorateur qui décrit la fonction comme une fonction noyau pour la réservation de vols.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:` : Définit une méthode pour réserver des vols avec des paramètres annotés et un type de retour.

La méthode construit des dictionnaires de paramètres pour les requêtes de vol aller et retour, et envoie des requêtes GET à l'API SERP. Elle vérifie le statut de la réponse et ajoute les informations de vol à la chaîne de résultats si la requête est réussie, ou imprime un message d'erreur si la requête a échoué. La méthode retourne la chaîne de résultats contenant les informations de vol.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Explication :
Importation des modules : Importer les modules nécessaires pour les identifiants Azure, l'agent IA, le contenu des messages de chat, le rôle de l'auteur et le décorateur de fonction du noyau.

Gestionnaire de contexte asynchrone : async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Cela configure un gestionnaire de contexte asynchrone pour gérer les identifiants Azure et créer un client d'agent IA.

Nom et instructions de l'agent : 
- `AGENT_NAME = "BookingAgent"` : Définit le nom de l'agent.
- `AGENT_INSTRUCTIONS = """..."""` : Fournit des instructions détaillées à l'agent sur la manière de traiter les demandes de réservation.

Créer la définition de l'agent : `agent_definition = await client.agents.create_agent(...)` : Crée une définition d'agent avec le modèle, le nom et les instructions spécifiés.

Créer l'agent AzureAI : `agent = AzureAIAgent(...)` : Crée un agent AzureAI en utilisant le client, la définition de l'agent et le plugin défini.

Créer un thread : `thread: AzureAIAgentThread | None = None` : Crée un thread pour l'agent. Il n'est pas nécessaire de créer un thread au préalable - si la valeur `None` est fournie, un nouveau thread sera créé lors de la première invocation et retourné dans la réponse.

Entrées utilisateur : `user_inputs = ["..."]` : Définit une liste d'entrées utilisateur que l'agent doit traiter.

Dans le bloc finally, supprimer le thread et l'agent pour libérer les ressources.


# Authentification

La classe `DefaultAzureCredential` fait partie du SDK Azure pour Python. Elle offre une méthode par défaut pour s'authentifier auprès des services Azure. Elle tente de s'authentifier en utilisant plusieurs méthodes dans un ordre spécifique, comme les variables d'environnement, l'identité managée et les identifiants Azure CLI.

Opérations asynchrones : Le module aio indique que la classe DefaultAzureCredential prend en charge les opérations asynchrones. Cela signifie que vous pouvez l'utiliser avec asyncio pour effectuer des requêtes d'authentification non bloquantes.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
